## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Busselton,-33.6500,115.3333,59.09,90,54,4.27,AU,2022-02-11 20:23:20,broken clouds
1,1,Aykhal,66.0000,111.5000,-7.69,100,99,5.61,RU,2022-02-11 20:28:23,overcast clouds
2,2,Harnosand,62.6323,17.9379,26.60,71,100,11.97,SE,2022-02-11 20:28:23,overcast clouds
3,3,Salalah,17.0151,54.0924,73.49,60,20,3.44,OM,2022-02-11 20:28:24,few clouds
4,4,Lompoc,34.6391,-120.4579,86.68,27,0,4.61,US,2022-02-11 20:28:24,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature for your trip? "))
max_temp = float(input("What is the maximum temperature for your trip? "))

What is the minimum temperature for your trip? 30
What is the maximum temperature for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
wanted_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
wanted_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Busselton,-33.6500,115.3333,59.09,90,54,4.27,AU,2022-02-11 20:23:20,broken clouds
3,3,Salalah,17.0151,54.0924,73.49,60,20,3.44,OM,2022-02-11 20:28:24,few clouds
4,4,Lompoc,34.6391,-120.4579,86.68,27,0,4.61,US,2022-02-11 20:28:24,clear sky
5,5,East London,-33.0153,27.9116,74.66,85,100,6.15,ZA,2022-02-11 20:23:18,light rain
7,7,Ushuaia,-54.8000,-68.3000,56.86,67,75,12.66,AR,2022-02-11 20:21:15,broken clouds
8,8,Hasaki,35.7333,140.8333,30.60,57,13,13.20,JP,2022-02-11 20:28:25,few clouds
9,9,Rikitea,-23.1203,-134.9692,79.03,73,4,16.28,PF,2022-02-11 20:23:38,clear sky
13,13,Beloha,-25.1667,45.0500,74.59,90,55,12.91,MG,2022-02-11 20:28:26,broken clouds
14,14,Ketchikan,55.3422,-131.6461,39.18,87,100,4.61,US,2022-02-11 20:28:27,overcast clouds
16,16,Hilo,19.7297,-155.0900,77.79,65,40,5.99,US,2022-02-11 20:24:41,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
wanted_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                6
Date                   0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = wanted_cities_df.dropna()
clean_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Busselton,-33.6500,115.3333,59.09,90,54,4.27,AU,2022-02-11 20:23:20,broken clouds
3,3,Salalah,17.0151,54.0924,73.49,60,20,3.44,OM,2022-02-11 20:28:24,few clouds
4,4,Lompoc,34.6391,-120.4579,86.68,27,0,4.61,US,2022-02-11 20:28:24,clear sky
5,5,East London,-33.0153,27.9116,74.66,85,100,6.15,ZA,2022-02-11 20:23:18,light rain
7,7,Ushuaia,-54.8000,-68.3000,56.86,67,75,12.66,AR,2022-02-11 20:21:15,broken clouds
...,...,...,...,...,...,...,...,...,...,...,...
698,698,Alta Floresta,-9.8756,-56.0861,82.22,78,40,4.61,BR,2022-02-11 20:47:08,scattered clouds
701,701,Comodoro Rivadavia,-45.8667,-67.5000,69.66,28,0,18.41,AR,2022-02-11 20:47:09,clear sky
702,702,Santa Maria,-29.6842,-53.8069,85.30,58,40,5.75,BR,2022-02-11 20:47:10,scattered clouds
703,703,Bantou,26.2151,117.4893,54.01,97,100,3.00,CN,2022-02-11 20:47:10,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,59.09,broken clouds,-33.6500,115.3333,
3,Salalah,OM,73.49,few clouds,17.0151,54.0924,
4,Lompoc,US,86.68,clear sky,34.6391,-120.4579,
5,East London,ZA,74.66,light rain,-33.0153,27.9116,
7,Ushuaia,AR,56.86,broken clouds,-54.8000,-68.3000,
8,Hasaki,JP,30.60,few clouds,35.7333,140.8333,
9,Rikitea,PF,79.03,clear sky,-23.1203,-134.9692,
13,Beloha,MG,74.59,broken clouds,-25.1667,45.0500,
14,Ketchikan,US,39.18,overcast clouds,55.3422,-131.6461,
16,Hilo,US,77.79,scattered clouds,19.7297,-155.0900,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel...skipping.")          

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot fin

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,59.09,broken clouds,-33.6500,115.3333,Observatory Guest House
3,Salalah,OM,73.49,few clouds,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
4,Lompoc,US,86.68,clear sky,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
5,East London,ZA,74.66,light rain,-33.0153,27.9116,Tu Casa
7,Ushuaia,AR,56.86,broken clouds,-54.8000,-68.3000,Albatross Hotel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))